In [97]:
import os
import pandas as pd
from sqlalchemy import create_engine

In [98]:
conn_str = os.environ["DB_WAREHOUSE_URI"]
read_sql_query = lambda sql: pd.read_sql_query(sql, conn_str)

# Scheduled loads

### All scheduled loads

In [99]:
query = """
SELECT *
FROM "factScheduledLoads"
"""
read_sql_query(query)

,Customer,CustomerCode,PONumber,PODate,ShippingNumber,LoadPieces,LoadTons
0,Trebor Inventory,221930,690810/16412,2024-06-10,68532.0,10.0,21.5455
1,Trebor Inventory,221930,690810/16412,2024-06-10,68533.0,10.0,20.0110
2,Trebor Inventory,282304,690714/12757,2024-05-14,68049.0,9.0,18.4030
3,Trebor Inventory,282304,690861/12840,2024-06-26,68682.0,9.0,18.2025
4,Trebor Inventory,282304,690861/12840,2024-06-26,68683.0,9.0,18.6920


### Q 100: How many loads are scheduled for PO # x?

#### Example: How many loads are scheduled for PO 690810/16412?

In [100]:
PONumber = "690810/16412"  # Update this to the PONumber you want to query

query = f"""
SELECT "PONumber", SUM("LoadPieces") AS "TotalPieces", SUM("LoadTons") AS "TotalTons"
FROM "factScheduledLoads"
WHERE "PONumber" = '{PONumber}'
GROUP BY "PONumber"
""".format(PONumber=PONumber)
read_sql_query(query)

,PONumber,TotalPieces,TotalTons
0,690810/16412,20.0,41.5565


### Q 101: How many loads are scheduled for customer code x?

#### Example: How many loads are scheduled for Customer Code 282304?

In [101]:
CustomerCode = "282304"  # Update this to the PONumber you want to query

query = f"""
SELECT "CustomerCode", SUM("LoadPieces") AS "TotalPieces", SUM("LoadTons") AS "TotalTons"
FROM "factScheduledLoads"
WHERE "CustomerCode" = '{CustomerCode}'
GROUP BY "CustomerCode"
""".format(CustomerCode=CustomerCode)
read_sql_query(query)

,CustomerCode,TotalPieces,TotalTons
0,282304,27.0,55.2975


### Q 25: Show the loads scheduled, by referencing pick up number with PO#

In [102]:
query = f"""
SELECT "PONumber", "ShippingNumber", "LoadPieces", "LoadTons"
FROM "factScheduledLoads"
"""
read_sql_query(query)

,PONumber,ShippingNumber,LoadPieces,LoadTons
0,690810/16412,68532.0,10.0,21.5455
1,690810/16412,68533.0,10.0,20.0110
2,690714/12757,68049.0,9.0,18.4030
3,690861/12840,68682.0,9.0,18.2025
4,690861/12840,68683.0,9.0,18.6920


### Q 26: Show the loads scheduled, by customer code

In [103]:
query = """
SELECT "CustomerCode", SUM("LoadPieces") AS "TotalPieces", SUM("LoadTons") AS "TotalTons"
FROM "factScheduledLoads"
GROUP BY "CustomerCode"
"""
read_sql_query(query)

,CustomerCode,TotalPieces,TotalTons
0,282304,27.0,55.2975
1,221930,20.0,41.5565


# Old inventory (non-shipped rolls)

### Q 22: Show the rolls, by customer code, that are 6 months to 1 year

In [104]:
start = (pd.Timestamp.now() - pd.DateOffset(months=12)).date()
end = (pd.Timestamp.now() - pd.DateOffset(months=6)).date()

query = f"""
SELECT *
FROM "factInventoryNotShipped"
WHERE "DateEntered" BETWEEN '{start}' AND '{end}'
ORDER BY "DateEntered"
""".format(start, end)
read_sql_query(query).head(10)

,RollID,WrapperNumber,DateEntered,TimeEntered,DateTimeEntered,RollNumber,SetPosition,GradeCode,CustomerGradeCode,Shift,...,RedPO,LabelPrinted,Breaks,NextSet,SlabDate,Workstation,LabelInitials,SCABarcode,SCAGrade,BayLocation
0,R5-23-09543C,WR5,2023-12-09,1899-12-30 07:59:00,2023-12-09 07:59:14,9543,C,125-98-1-090,282304,Day,...,None,-1,NaN,D,None,STT-CUCONWRAPPE,MW,None,None,None
1,R5-23-09556B,WR5,2023-12-09,1899-12-30 18:51:00,2023-12-09 18:51:48,9556,B,125-98-1-090,282326,Night,...,None,-1,1.0,C,None,STT-CUCONWRAPPE,AB,None,None,None
2,R5-23-09557B,WR5,2023-12-09,1899-12-30 18:51:00,2023-12-09 18:51:45,9557,B,125-98-1-090,282326,Night,...,None,-1,NaN,C,None,STT-CUCONWRAPPE,AB,None,None,None
3,R5-23-09607A,WR5,2023-12-11,1899-12-30 21:27:00,2023-12-11 21:27:48,9607,A,125-98-1-090,282326,Night,...,None,-1,NaN,B,12/14/2023,STT-CUCONWRAPPE,TC,None,None,None
4,R5-23-09615A,WR5,2023-12-11,1899-12-30 05:50:00,2023-12-12 05:50:56,9615,A,125-98-1-090,282326,Night,...,None,-1,2.0,B,12/14/2023,STT-CUCONWRAPPE,TC,None,None,None
5,R6-23-10816B,WR6,2023-12-13,1899-12-30 02:54:00,2023-12-14 02:54:25,10816,B,200-98-3-100,917298,Night,...,None,-1,NaN,C,None,STT-WRAPPER6,AJ,None,None,None
6,R6-23-10816C,WR6,2023-12-13,1899-12-30 02:54:00,2023-12-14 02:54:28,10816,C,200-98-3-100,917298,Night,...,None,-1,NaN,D,None,STT-WRAPPER6,AJ,None,None,None
7,R5-23-09666A,WR5,2023-12-14,1899-12-30 02:57:00,2023-12-15 02:57:33,9666,A,125-98-1-090,959126,Night,...,None,-1,NaN,B,12/20/2023,STT-CUCONWRAPPE,MW,None,756807,None
8,R5-23-09663A,WR5,2023-12-14,1899-12-30 02:55:00,2023-12-15 02:55:57,9663,A,125-98-1-090,282326,Night,...,None,-1,NaN,B,12/20/2023,STT-CUCONWRAPPE,MW,None,None,None
9,R5-23-09664A,WR5,2023-12-14,1899-12-30 02:57:00,2023-12-15 02:57:25,9664,A,125-98-1-090,282326,Night,...,None,-1,NaN,B,12/20/2023,STT-CUCONWRAPPE,MW,None,None,None


### Q 23: Show the rolls, by customer code, that are 1-2 years old

In [105]:
start = (pd.Timestamp.now() - pd.DateOffset(months=24)).date()
end = (pd.Timestamp.now() - pd.DateOffset(months=12)).date()

query = f"""
SELECT *
FROM "factInventoryNotShipped"
WHERE "DateEntered" BETWEEN '{start}' AND '{end}'
ORDER BY "DateEntered"
""".format(start, end)
read_sql_query(query).head(10)

,RollID,WrapperNumber,DateEntered,TimeEntered,DateTimeEntered,RollNumber,SetPosition,GradeCode,CustomerGradeCode,Shift,...,RedPO,LabelPrinted,Breaks,NextSet,SlabDate,Workstation,LabelInitials,SCABarcode,SCAGrade,BayLocation
0,R6-22-13384B,WR6,2022-12-11,1899-12-30 05:15:00,2022-12-12 05:15:56,13384,B,190-98-3-110,971900,Night,...,None,-1,1.0,C,None,STT-WRAPPER6,WJ,None,None,None
1,R5-22-09274A,WR5,2022-12-11,1899-12-30 00:30:00,2022-12-12 00:30:46,9274,A,125-98-1-090,289999,Night,...,None,-1,NaN,B,12/11/2022,STT-CUCONWRAPPE,TC,None,None,None
2,R6-22-13576A,WR6,2022-12-16,1899-12-30 21:38:00,2022-12-16 21:38:52,13576,A,200-98-3-100,532002,Night,...,None,-1,NaN,B,None,STT-WRAPPER6,AE,None,None,None
3,R5-22-09410C,WR5,2022-12-16,1899-12-30 19:41:00,2022-12-16 19:41:43,9410,C,125-98-1-090,959999,Night,...,None,-1,NaN,D,12/16/2022,STT-CUCONWRAPPE,JL,None,756088,None
4,R5-22-09411A,WR5,2022-12-16,1899-12-30 19:43:00,2022-12-16 19:43:32,9411,A,125-98-1-090,959999,Night,...,None,-1,NaN,B,12/16/2022,STT-CUCONWRAPPE,JL,None,756088,None
5,R5-22-09429B,WR5,2022-12-17,1899-12-30 10:59:00,2022-12-17 10:59:10,9429,B,125-98-1-090,280000,Day,...,None,-1,NaN,C,12/17/2022,STT-CUCONWRAPPE,TC,None,None,None
6,R6-22-13670B,WR6,2022-12-18,1899-12-30 04:25:00,2022-12-19 04:25:47,13670,B,195-98-3-100,351999,Night,...,None,-1,NaN,C,None,STT-WRAPPER6,AE,None,None,None
7,R6-22-13682C,WR6,2022-12-19,1899-12-30 12:00:00,2022-12-19 12:00:08,13682,C,195-98-3-100,351999,Day,...,None,-1,NaN,D,None,STT-WRAPPER6,AJ,None,None,None
8,R5-22-09582B,WR5,2022-12-22,1899-12-30 00:55:00,2022-12-23 00:55:08,9582,B,125-98-1-090,280000,Night,...,None,-1,NaN,C,12/22/2022,STT-CUCONWRAPPE,AB,None,None,None
9,R6-22-13866C,WR6,2022-12-24,1899-12-30 02:39:00,2022-12-25 02:39:57,13866,C,210-98-3-100,829855,Night,...,None,-1,1.0,D,None,STT-WRAPPER6,JW,None,None,None


### Q 24: show the rolls, by customer code, that are 2 years and older

In [106]:
start = (pd.Timestamp.now() - pd.DateOffset(months=24)).date()

query = f"""
SELECT *
FROM "factInventoryNotShipped"
WHERE "DateEntered" <= '{start}'
ORDER BY "DateEntered"
""".format(start, end)
read_sql_query(query).head(10)

,RollID,WrapperNumber,DateEntered,TimeEntered,DateTimeEntered,RollNumber,SetPosition,GradeCode,CustomerGradeCode,Shift,...,RedPO,LabelPrinted,Breaks,NextSet,SlabDate,Workstation,LabelInitials,SCABarcode,SCAGrade,BayLocation
0,R6-13-3212A-D,WR6,2013-08-11,NaT,2013-08-11,3212,A,220-98-3-100,780160,None,...,None,0,0.0,None,None,None,None,None,None,None
1,R6-15-28308D,WR6,2015-09-19,NaT,2015-09-19,28308,D,125-98-1-100,673013,Day,...,None,0,0.0,None,None,None,None,None,None,A10
2,R6-15-28309D,WR6,2015-09-19,NaT,2015-09-19,28309,D,125-98-1-100,673013,Day,...,None,0,0.0,None,None,None,None,None,None,A10
3,R6-15-28307D,WR6,2015-09-19,NaT,2015-09-19,28307,D,125-98-1-100,673013,Day,...,None,0,0.0,None,None,None,None,None,None,A10
4,R6-15-28310D,WR6,2015-09-19,NaT,2015-09-19,28310,D,125-98-1-100,673013,Day,...,None,0,0.0,None,None,None,None,None,None,A10
5,R6-15-28311D,WR6,2015-09-19,NaT,2015-09-19,28311,D,125-98-1-100,673013,Day,...,None,0,0.0,None,None,None,None,None,None,A10
6,R6-15-28312D,WR6,2015-09-20,NaT,2015-09-20,28312,D,125-98-1-100,673013,Day,...,None,0,0.0,None,None,None,None,None,None,A10
7,R6-16-52594E,WR6,2016-12-15,NaT,2016-12-15,52594,E,200-98-3-100,910300,Day,...,None,0,0.0,None,None,None,None,None,None,818
8,R6-16-52594D,WR6,2016-12-15,NaT,2016-12-15,52594,D,200-98-3-100,910300,Day,...,None,0,0.0,None,None,None,None,None,None,818
9,R6-16-52593B,WR6,2016-12-15,NaT,2016-12-15,52593,B,200-98-3-100,910315,Day,...,None,0,0.0,None,None,None,None,None,None,819


### Q 102: Inventory by age by customer

In [107]:
customer = "Trebor Inventory"

query = f"""
SELECT *
FROM "factInventoryNotShipped"
WHERE "Customer" = '{customer}'
ORDER BY "DateEntered"
""".format(customer)
read_sql_query(query).head(10)

,RollID,WrapperNumber,DateEntered,TimeEntered,DateTimeEntered,RollNumber,SetPosition,GradeCode,CustomerGradeCode,Shift,...,RedPO,LabelPrinted,Breaks,NextSet,SlabDate,Workstation,LabelInitials,SCABarcode,SCAGrade,BayLocation
0,R5-20-04884C,WR5,2020-06-19,1899-12-30 01:41:00,2020-06-20 01:41:17,4884,C,115-76-3-121,227600,Night,...,None,0,NaN,D,None,STT-CUCONWRAPPE,None,None,None,None
1,R5-20-04979B,WR5,2020-06-22,1899-12-30 03:00:00,2020-06-23 03:00:47,4979,B,115-76-3-121,220030,Night,...,None,0,NaN,C,None,STT-CUCONWRAPPE,None,None,None,None
2,R5-20-05027C,WR5,2020-06-24,1899-12-30 02:59:00,2020-06-25 02:59:09,5027,C,115-76-3-121,227600,Night,...,None,-1,NaN,D,None,STT-CUCONWRAPPE,BJ,None,None,None
3,R5-20-05204C,WR5,2020-06-30,1899-12-30 00:01:00,2020-07-01 00:01:58,5204,C,115-76-3-121,227600,Night,...,None,-1,NaN,D,None,STT-CUCONWRAPPE,JB,None,None,None
4,R5-20-05204A,WR5,2020-06-30,1899-12-30 00:01:00,2020-07-01 00:01:52,5204,A,115-76-3-121,227600,Night,...,None,-1,NaN,B,None,STT-CUCONWRAPPE,JB,None,None,None
5,R5-20-05204B,WR5,2020-06-30,1899-12-30 00:01:00,2020-07-01 00:01:55,5204,B,115-76-3-121,220030,Night,...,None,-1,NaN,C,None,STT-CUCONWRAPPE,JB,None,None,None
6,R5-20-05350B,WR5,2020-07-05,1899-12-30 22:06:00,2020-07-05 22:06:19,5350,B,115-76-3-121,220030,Night,...,None,-1,NaN,C,None,STT-CUCONWRAPPE,JK,None,None,None
7,R5-20-05374A,WR5,2020-07-06,1899-12-30 14:51:00,2020-07-06 14:51:16,5374,A,115-76-3-121,227600,Day,...,None,-1,NaN,B,None,STT-CUCONWRAPPE,JB,None,None,None
8,R5-20-05507A,WR5,2020-07-10,1899-12-30 21:27:00,2020-07-10 21:27:31,5507,A,115-76-3-121,227600,Night,...,None,-1,NaN,B,None,STT-CUCONWRAPPE,BJ,None,None,None
9,R5-20-05506A,WR5,2020-07-10,1899-12-30 21:26:00,2020-07-10 21:26:41,5506,A,115-76-3-121,227600,Night,...,None,-1,1.0,B,None,STT-CUCONWRAPPE,BJ,None,None,None
